# Example of a schedview combined event + visit timeline

In [ ]:
night = '2024-12-08'
telescope = 'Simonyi'
visit_origin = "lsstcomcam"

In [ ]:
# Use more recent versions of rubin_scheduler and schedview (will become unnecessary soon)
import sys
extra_paths = (
    "/sdf/data/rubin/shared/scheduler/packages/rubin_scheduler-3.4.1.dev2+g9f70dc6.d20241204",
    "/sdf/data/rubin/shared/scheduler/packages/rubin_sim-2.1.1.dev71+g0a9c0ea9",
    "/sdf/data/rubin/shared/scheduler/packages/SP-1789/schedview",
)
for extra_path in extra_paths:
    sys.path.insert(0, extra_path)

import os
import astropy.utils.iers
import bokeh.io
import schedview.dayobs
import schedview.collect.nightreport
import schedview.collect.visits
import schedview.collect.efd
import schedview.compute.astro
import schedview.compute.obsblocks
import schedview.compute.nightreport
import schedview.plot.timeline
import schedview.collect.timeline

In [ ]:
# Tell astropy we don't need super high precision
astropy.utils.iers.conf.iers_degraded_accuracy = "ignore"

In [ ]:
# Tell rubin_scheduler where to find its data
os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/shared/rubin_sim_data"

In [ ]:
# Prepare jupyter to show bokeh plots
bokeh.io.output_notebook(hide_banner=True)

In [ ]:
# Get the DayObs instance from whatever form the user
# provided it in.

day_obs = schedview.dayobs.DayObs.from_date(night)

In [ ]:
# Collect data from outside sources of data such as
# the consdb or the EFD.

data = await schedview.collect.timeline.collect_timeline_data(
    day_obs,
    sal_indexes=schedview.collect.efd.SAL_INDEX_GUESSES[visit_origin],
    telescope=telescope,
    visit_origin=visit_origin,
    visits=True,
    log_messages=True,
    scheduler_dependencies=True,
    scheduler_configuration=True,
    block_status=True,
    scheduler_snapshots=True
)

In [ ]:
# Things to plot that are not collected from outside data sources directly,
# but rather either computed from them or derived from models.

data['block_spans'] = schedview.compute.obsblocks.compute_block_spans(data['block_status'])
data['visit_timeline'] = data['visits']
data['model_sky'] = schedview.compute.astro.get_median_model_sky(day_obs)
data['sun'] = schedview.compute.astro.night_events(day_obs.date)

In [ ]:
# Actually make the plot

p = schedview.plot.timeline.make_timeline_scatterplots(**data)
bokeh.io.show(p)

You can make a full stand-alone (*not* using jupyter or Times Square) dashboard web application using `panel` like this:

In [ ]:
import inspect
from schedview.examples.app import combined_timeline
print(inspect.getsource(combined_timeline))